In [0]:
import pyspark.sql.functions as F

In [0]:
df_sellout_with_dups = spark.table("d_dnst_030_gold.pbi_views.v_sellout_customers_with_duplicates")

In [0]:
(
    df_sellout_with_dups
    .filter(F.col("Id at Wholesaler").isin("KB09140", "SPBEKB09140"))
    .orderBy(F.col("Id at Wholesaler"))
    .display()
)

In [0]:
(
    df_sellout_with_dups
    .filter(F.col("Id at Wholesaler").isin("KB09140", "SPBEKB09140"))
    .select(
        # "Id at Wholesaler", # Include this for ordering
        "dim_sellout_cust_id", 
        "C4s Account Id"
    )
    .distinct() # Or .dropDuplicates(["dim_sellout_cust_id", "C4s Account Id"])
    # .orderBy(F.col("Id at Wholesaler"))
    .display()
)

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Define the columns that determine uniqueness (the partition key)
# The row with RN=1 will be selected as the survivor for each unique combination of these two IDs.
window_spec = Window.partitionBy("dim_sellout_cust_id", "C4s Account Id").orderBy(F.lit(1))

df_deduplicated = (
    df_sellout_with_dups
    .filter(F.col("Id at Wholesaler").isin("KB09140", "SPBEKB09140"))
    .withColumn(
        "RN", 
        F.row_number().over(window_spec)
    )
    .filter(F.col("RN") == 1)
    .drop("RN") # Remove the temporary rank column
)

# Display the resulting DataFrame with all columns deduplicated on the specified IDs
df_deduplicated.display()